# Policy Gradients

## 1. Creating the Environment

In [3]:
import gym
env = gym.make('CartPole-v0')

episodes = 100
iters = 1000
render_step = 10

for episode in range(episodes):
    observation = env.reset()
    for i in range(iters):
        if episode * i % render_step == 0:
            env.render()
        env.step(env.action_space.sample()) # take a random action


You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.


ArgumentError: argument 2: <class 'TypeError'>: wrong type